In [2]:
!pip install folium

In [3]:
!pip install openpyxl

In [4]:
import pandas as pd
import requests
import folium
import re

address_file = r'고등교육기관 주소록(2024.4.1.).xlsx'
address_df = pd.read_excel(address_file, engine='openpyxl')
address_df.columns = address_df.loc[4].tolist() 
address_df = address_df.drop(index=list(range(0, 5))) 
address_df = address_df[['학교명', '주소']]  
address_df.columns = ['이름', '주소'] 

heatwave_file = r'지역별 폭염일수.xlsx'
heatwave_df = pd.read_excel(heatwave_file, engine='openpyxl')
heatwave_df.columns = ['이름', '경도', '위도', '폭염일수']

merged_df = pd.merge(heatwave_df, address_df, on='이름', how='left')

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type=ROAD'
primary_key = '2A645740-77BD-380B-81D8-8AB51C078AB8'

def request_geo(address):
    """주소를 좌표(경도, 위도)로 변환"""
    full_url = f"{url}&address={address}&key={primary_key}"
    response = requests.get(full_url)
    data = response.json()
    if data['response']['status'] == 'OK':
        x = float(data['response']['result']['point']['x']) 
        y = float(data['response']['result']['point']['y'])  
        return x, y
    else:
        return None, None

longitude_list = []
latitude_list = []

for addr in merged_df['주소']:
    if pd.notna(addr): 
        clean_addr = re.sub(r'\([^)]*\)', '', addr)  
        x, y = request_geo(clean_addr)
        longitude_list.append(x if x is not None else 0) 
        latitude_list.append(y if y is not None else 0)
    else:
        longitude_list.append(0)  
        latitude_list.append(0)

merged_df['경도'] = longitude_list
merged_df['위도'] = latitude_list

center_lat = merged_df['위도'].mean()
center_lon = merged_df['경도'].mean()
map = folium.Map(location=[center_lat, center_lon], zoom_start=7)

for i in range(len(merged_df)):
    if merged_df['경도'][i] != 0 and merged_df['위도'][i] != 0: 
        folium.CircleMarker(
            location=[merged_df['위도'][i], merged_df['경도'][i]],
            radius=merged_df['폭염일수'][i],  
            popup=f"{merged_df['이름'][i]}: {merged_df['폭염일수'][i]}일",
            color='orange',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(map)

output_path = r'heatwave_map_with_geo.html'
map.save(output_path)
print(f"지도 생성 완료: {output_path}")

지도 생성 완료: heatwave_map_with_geo.html


In [5]:
!pip install qrcode

In [6]:
import qrcode

github_url = "https://github.com/kasung1020/kangsungbin"  


qr = qrcode.QRCode(
    version=1,  
    error_correction=qrcode.constants.ERROR_CORRECT_L,  # 오류 보정 수준
    box_size=10,  
    border=4, 
)
qr.add_data(github_url)
qr.make(fit=True)

img = qr.make_image(fill_color="black", back_color="white")
img_path = "github_qr_code.png"
img.save(img_path)

print(f"QR 코드 생성 완료: {img_path}")

QR 코드 생성 완료: github_qr_code.png
